## Jonathan Perkins
## 4/18/2024
## PH 418
## HW 6

# Feature Generation

We will create machine learning models to predict material properties solely based on their
chemical compositions. The first step is to create input features (or descriptors or attributes).
We will follow the paper by Ward et al. in npj Computational Materials (2016) 2, 16028.
The features are designed to enable machine learning algorithms to construct general rules
that can possibly learn and reflect to some extent physical and chemical intuitions.
Please write a python function with a single string variable of chemical formula (such as
‘Fe2O3’) as the input argument. Upon completion, the function will return a list containing
the following 60 features (floating point or integer values)

# Imports

In [1]:
import pymatgen.core as pmg

ModuleNotFoundError: No module named 'pymatgen'